In [9]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from datetime import datetime, timedelta
import cartopy.feature as cfeat
import xarray as xr
from datetime import datetime
SHP='../Data/TPBoundary_new/TPBoundary_new.shp'

In [23]:
# 将字符串转换为日期对象，再转换成制定格式
select_time="2020-08-27"
select_time2 = datetime.strptime(select_time, "%Y-%m-%d").strftime("%Y%m%d")


uwnd_path="../Data/uwnd.10m.gauss.2020.nc"
vwnd_path="../Data/vwnd.10m.gauss.2020.nc"
prate_path="../Data/Pre2020/tpmfd_prcp_d_"+select_time2+".nc"
uwnd_data=xr.open_dataset(uwnd_path)
vwnd_data=xr.open_dataset(vwnd_path)
prate_data=xr.open_dataset(prate_path)

#查看prcp变量维度，和第一时次的数据
# print(prate_data.dims)
# print(prate_data)
prcp=prate_data["prcp"]
prcp_lon=prate_data["longitude"]
prcp_lat=prate_data["latitude"]

uwnd=uwnd_data["uwnd"]
vwnd=vwnd_data["vwnd"]

#选特定的时间的变量
select_uwnd=uwnd.sel(time=select_time)
select_vwnd=vwnd.sel(time=select_time)
# print(type(select_uwnd))                        ##<class 'xarray.core.dataarray.DataArray'>
# print(type(select_uwnd.values))                 ##<class 'numpy.ndarray'>

#找到属于风的经纬度
wnd_lon=uwnd_data["lon"]
wnd_lat=uwnd_data["lat"]

In [24]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(6, 6))
#resolution='i',

mymap = Basemap( projection='cyl', llcrnrlon=67, urcrnrlon=105, llcrnrlat=26, urcrnrlat=41)


# mymap.drawmeridians(np.arange(67, 105, 15), labels=[False, False, False, True])  # Labels on right side
# mymap.drawparallels(np.arange(26, 41, 2), labels=[True, True, False, False])  # Labels on top

mymap.drawmeridians(np.arange(68, 106, 6), labels=[False, False, False,True])  # Labels on right side
mymap.drawparallels(np.arange(26, 41, 2), labels=[True,  False])  # Labels on top

mymap.etopo()
mymap.bluemarble()
mymap.shadedrelief()
mymap.drawcoastlines()
#画风场
axqui=mymap.quiver(wnd_lon,wnd_lat,select_uwnd.values,select_vwnd.values,pivot='mid',\
                    width=0.003,scale=110,color='k',angles='xy',zorder=1)
ax = plt.gca()   
ax.set_title(f"{select_time}"+" Wind Field", loc='left') 
ax.quiverkey(axqui,0.85, 1.05,5, '5 m/s', labelpos='E') 
plt.text(90, 29.9, 'NAMCO', fontsize=12, ha='center', va='center', color='r')
plt.scatter(91,31,marker='^',color='red')
plt.savefig(f"../../Figures/{select_time}地形风场.png", dpi=600, bbox_inches='tight')
plt.show()